In [2]:
import tabula
import pandas as pd

In [3]:
filenames = ['best_sector_times_64.pdf','official_starting_grid_41.pdf',
'fastest_laps_24.pdf',
'history_chart_25.pdf','pit_stop_summary_29.pdf',
'lap_analysis_25.pdf','provisional_starting_grid_37.pdf',
'lap_chart_31.pdf','speed_trap_45.pdf',
'maximum_speeds_62.pdf']

In [4]:
#tabula.read_pdf("pit_stop_summary_29.pdf")

In [5]:
'fastest_laps_24.pdf'[:-6]+'df'

'fastest_laps_df'

In [8]:
for pidieffe in filenames:
    #print(pidieffe)
    frame_name = pidieffe[:-6]+'df'
    #print(frame_name)
    exec( frame_name+" = tabula.read_pdf(pidieffe)")
    #exec(frame_name+".head()")

In [27]:
'pit_stop_summary_29.pdf'[:-6]+'df'

'pit_stop_summary_df'

In [10]:
pit_stop_summary_df

,NO,DRIVER,ENTRANT,LAP,TIME OF DAY,S T OP,Unnamed: 6,DURATION,TOTAL TIME
0,27,N. HULKENBERG,Renault Sport F1 Team,1,14:05:25,1,NaN,23.550,23.550
1,5,S. VETTEL,Scuderia Ferrari,2,14:07:35,1,NaN,22.906,22.906
2,26,D. KVYAT,Scuderia Toro Rosso,2,14:07:46,1,NaN,24.016,24.016
3,14,F. ALONSO,McLaren Honda,2,14:07:48,1,NaN,24.250,24.250
4,19,F. MASSA,Williams Martini Racing,2,14:07:51,1,NaN,23.123,23.123
5,11,S. PEREZ,Sahara Force India F1 Team,2,14:07:52,1,NaN,24.030,24.030
6,20,K. MAGNUSSEN,Haas F1 Team,2,14:07:53,1,NaN,23.809,23.809
7,9,M. ERICSSON,Sauber F1 Team,2,14:07:54,1,NaN,27.934,27.934
8,31,E. OCON,Sahara Force India F1 Team,2,14:07:57,1,NaN,17.673,17.673
9,8,R. GROSJEAN,Haas F1 Team,2,14:08:00,1,NaN,30.654,30.654


In [8]:
df.describe()

,NO,LAP,S T OP,Unnamed: 6,DURATION,TOTAL TIME
count,27.000000,27.000000,27.000000,0.0,27.000000,27.000000
mean,21.666667,3.074074,1.296296,NaN,22.427259,29.277370
std,17.453234,1.071517,0.465322,NaN,5.082608,9.617541
min,2.000000,1.000000,1.000000,NaN,17.291000,17.291000
25%,8.500000,2.000000,1.000000,NaN,17.338500,23.097000
50%,19.000000,4.000000,1.000000,NaN,23.071000,24.250000
75%,30.500000,4.000000,2.000000,NaN,24.140000,40.353500
max,77.000000,4.000000,2.000000,NaN,33.491000,51.164000
